### Experiment 0


The objective with this experiment is to test the pipeline devloped.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
import datetime
import os
import numpy as np
import tensorflow as tf
import keras
import sklearn.model_selection as sk
import copy
from PIL import Image as IM
from keras.callbacks import TensorBoard
import cv2
import json
import pathlib
from glob import glob

Using TensorFlow backend.


In [2]:
## import helpers
from helpers import data_loaders as dls
from helpers import pointcloud as pc
from helpers.viz import plot
from helpers.logger import Logger

In [3]:
PATH = '../' # path of the repo.
_NAME = 'experiment0' #name of experiment
!ls $PATH

dataset  nbs  README.md  TODO


In [4]:
_EXP, _LOG, _TMP, _MODEL, _RES = dls.create_dir_struct(PATH, _NAME)
logger = Logger('EXP0', _LOG + 'experiment0.log')
logger.debug('Logger EXP0 int')
!ls $_EXP

log  result  tmp


In [5]:
# get dataset
train_set, valid_set, test_set = dls.get_dataset(PATH, is_training=True)